<a href="https://colab.research.google.com/github/ryogrid/ryogridJupyterNotebooks/blob/master/gpt_index_trial_roberta_230611_2111.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/gpt-index-trial

/content/drive/MyDrive/gpt-index-trial


In [ ]:
!cat requirements.txt

llama-index
langchain
transformers
sentence-transformers
sentencepiece

In [3]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.5/493.5 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 108.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 111.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 91.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 78.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 1

In [4]:
from llama_index import LLMPredictor
from langchain.llms.base import LLM
from typing import Optional, List, Mapping, Any

In [ ]:
!pwd
!ls

/content/drive/MyDrive/gpt-index-trial
requirements.txt


In [80]:
DEFAULT_PROMPT = """
文脈情報は以下です。
---
{context_str}
---
事前知識ではなく、文脈情報を参考に質問に答えてください。：{query_str}
"""

REFINE_PROMPT = """
質問は以下です。：{query_str}
すでに答えの候補があります。：{existing_answer}
必要な場合のみ、以下の情報を使って上の答えを改良することができます。文脈情報が有用でない場合は元の答えをそのまま返してください。
---
{context_msg}
---
"""

In [21]:
import torch
from transformers import AutoTokenizer, LukeForQuestionAnswering, T5Tokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Mizuiro-sakura/luke-japanese-large-finetuned-QA")
qa_model = LukeForQuestionAnswering.from_pretrained("Mizuiro-sakura/luke-japanese-large-finetuned-QA")

#if torch.cuda.is_available():
    #qa_model = qa_model.to("cuda")
    #qa_model0 = qa_model0.to("cuda")


In [81]:
import numpy as np

def inference_answer(prompt):
    #print(prompt)
    splited = prompt.split("文脈情報を参考に質問に答えてください。：")
    #print(len(splited))
    #print(splited)
    if len(splited) > 1:      
      question = splited[1]
      context = prompt.replace(question, "")
    else:
      # リファインする場合
      #splited = prompt.split("必要な場合のみ、以下の文脈情報を使ってこの答えを改良することができます。")
      #question = splited[0] + "必要な場合のみ、以下の文脈情報を使ってこの答えを改良することができます。"
      question = prompt
      #context = prompt.replace(question,"")
      context = ""

    print("Q:" + question + "\n")
    print("Context:" + context + "\n")
    test_feature = tokenizer(
        question,
        context,
        max_length=318,
        truncation=True
    )
    with torch.no_grad():
        outputs = qa_model(torch.tensor([test_feature["input_ids"]]))
    #start_logits = outputs.start_logits.cpu().numpy()
    #end_logits = outputs.end_logits.cpu().numpy()
    start_logits = outputs.start_logits.to(qa_model.device).numpy()
    end_logits = outputs.end_logits.to(qa_model.device).numpy()
    answer_ids = test_feature["input_ids"][np.argmax(start_logits):np.argmax(end_logits)+1]
    return "".join(tokenizer.batch_decode(answer_ids))

In [ ]:
def generate(prompt):
    token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
    n = len(token_ids[0])

    with torch.no_grad():
        output_ids = qa_model.generate(
            token_ids.to(qa_model.device),
            max_length=n+100,
            min_length=n+2,
            do_sample=False,
            pad_token_id=tokenizer.pad_token_id,
            bos_token_id=tokenizer.bos_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    output = tokenizer.decode(output_ids.tolist()[0][n:])
    return output.replace("</s>", "")

In [23]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import LangchainEmbedding

# load in HF embedding model from langchain
embed_model = LangchainEmbedding(HuggingFaceEmbeddings(model_name="oshizo/sbert-jsnli-luke-japanese-base-lite"))
embed_model._langchain_embedding.client.max_seq_length = 256

#if torch.cuda.is_available():
#  embed_model.to("cuda")

In [24]:
class CustomLLM(LLM):
        
    @property
    def _llm_type(self) -> str:
        return "custom"
    
    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        return inference_answer(prompt)
    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        return {"name":"custom"}
llm = CustomLLM()
llm_predictor = LLMPredictor(llm=llm)

In [11]:
import urllib
import json
url = "https://ja.wikipedia.org/wiki/%E3%81%BC%E3%81%A3%E3%81%A1%E3%83%BB%E3%81%96%E3%83%BB%E3%82%8D%E3%81%A3%E3%81%8F!?action=cirrusdump"
with urllib.request.urlopen(url) as f:
    data = f.read()
text = json.loads(data)[0]["_source"]["text"]

In [70]:
from typing import Dict
from llama_index.callbacks.base import BaseCallbackHandler
from llama_index.callbacks.schema import CBEventType

class CustomCallbackHandler(BaseCallbackHandler):

    def __init__(
        self,
        event_starts_to_ignore: Optional[List[CBEventType]] = None,
        event_ends_to_ignore: Optional[List[CBEventType]] = None,
        print_trace_on_end: bool = True,
    ) -> None:
        event_starts_to_ignore = event_starts_to_ignore if event_starts_to_ignore else []
        event_ends_to_ignore = event_ends_to_ignore if event_ends_to_ignore else []
        super().__init__(
            event_starts_to_ignore=event_starts_to_ignore,
            event_ends_to_ignore=event_ends_to_ignore,
        )

    def on_event_start(
        self,
        event_type: CBEventType,
        payload: Optional[Dict[str, Any]] = None,
        event_id: str = "",
        **kwargs: Any
    ) -> str:
        print(f"event_type = {event_type} (start)")
        return event_id

    def on_event_end(
        self,
        event_type: CBEventType,
        payload: Optional[Dict[str, Any]] = None,
        event_id: str = "",
        **kwargs: Any
    ) -> None:
        print(f"event_type = {event_type} (end)")
     #   if event_type == CBEventType.RETRIEVE:
     #     for selected_node in payload["nodes"]:
     #       print(selected_node)

    def start_trace(self, trace_id: Optional[str] = None) -> None:
        print("start_trace")

    def end_trace(
        self,
        trace_id: Optional[str] = None,
        trace_map: Optional[Dict[str, List[str]]] = None,
    ) -> None:
        print("end_trace")

In [71]:
#from llama_index import GPTVectorStoreIndex
from llama_index import LLMPredictor, ServiceContext, GPTSimpleKeywordTableIndex, SimpleKeywordTableIndex, VectorStoreIndex
from llama_index.readers.schema.base import Document
from llama_index.indices.list.base import ListRetrieverMode
from llama_index.callbacks import CallbackManager, LlamaDebugHandler
from llama_index.indices.service_context import set_global_service_context
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.indices.query.response_synthesis import ResponseSynthesizer
from llama_index.indices.response.type import ResponseMode
from llama_index.indices.prompt_helper import PromptHelper

documents = []
for i in range(0, len(text), 200):
    documents.append(Document(text[i:i+200]))
    if i != 0:
        documents.append(Document(text[i-100:i+100]))

#llm_predictor   = LLMPredictor(llm=llm)
llama_debug_handler = LlamaDebugHandler()
custom_callback_handler = CustomCallbackHandler()
callback_manager = CallbackManager([
    llama_debug_handler
    , custom_callback_handler
])
#callback_manager = CallbackManager([llama_debug_handler])

service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, callback_manager=callback_manager, prompt_helper=PromptHelper.from_llm_metadata(llm_predictor.get_llm_metadata()), embed_model=embed_model)
#set_global_service_context(service_context=service_context)
#service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor)

# インデックスの作成
#index = GPTVectorStoreIndex.from_documents(documents, llm_predictor=llm_predictor, embed_model=embed_model)
#index = GPTSimpleKeywordTableIndex.from_documents(documents, service_context=service_context, llm_predictor=llm_predictor, embed_model=embed_model)
index = VectorStoreIndex.from_documents(documents, service_context=service_context, llm_predictor=llm_predictor, embed_model=embed_model)

start_trace
event_type = node_parsing (start)
event_type = chunking (start)
event_type = chunking (end)
event_type = chunking (start)
event_type = chunking (end)
event_type = chunking (start)
event_type = chunking (end)
event_type = chunking (start)
event_type = chunking (end)
event_type = chunking (start)
event_type = chunking (end)
event_type = chunking (start)
event_type = chunking (end)
event_type = chunking (start)
event_type = chunking (end)
event_type = chunking (start)
event_type = chunking (end)
event_type = chunking (start)
event_type = chunking (end)
event_type = chunking (start)
event_type = chunking (end)
event_type = chunking (start)
event_type = chunking (end)
event_type = chunking (start)
event_type = chunking (end)
event_type = chunking (start)
event_type = chunking (end)
event_type = chunking (start)
event_type = chunking (end)
event_type = chunking (start)
event_type = chunking (end)
event_type = chunking (start)
event_type = chunking (end)
event_type = chunking (sta

In [ ]:
len(documents)

253

In [84]:
from llama_index.prompts.base import Prompt
from llama_index.prompts.prompts import RefinePrompt, QuestionAnswerPrompt
refine_prompt = RefinePrompt(REFINE_PROMPT)
default_prompt = QuestionAnswerPrompt(DEFAULT_PROMPT)

In [100]:
#query_engine = index.as_query_engine()

'''
retriever = VectorIndexRetriever(
    index=index, 
    similarity_top_k=2,
)
'''

retriever = index.as_retriever(retriever_mode='embedding',  similarity_top_k=5)
#retriever = index.as_retriever(retriever_mode='simple')
#retriever = index.as_retriever()
#retriever = index.as_retriever(retriever_mode='embedding')

#response_synthesizer = ResponseSynthesizer.from_args(response_mode = ResponseMode.REFINE ,service_context=service_context, text_qa_template=default_prompt, refine_template=refine_prompt, callback_manager=callback_manager, verbose=True)
#response_synthesizer = ResponseSynthesizer.from_args(response_mode = ResponseMode.COMPACT ,service_context=service_context, text_qa_template=default_prompt, refine_template=refine_prompt, callback_manager=callback_manager, verbose=True)
response_synthesizer = ResponseSynthesizer.from_args(response_mode = ResponseMode.SIMPLE_SUMMARIZE, service_context=service_context, text_qa_template=default_prompt, refine_template=refine_prompt, callback_manager=callback_manager, verbose=True)
query_engine = RetrieverQueryEngine(retriever=retriever, response_synthesizer=response_synthesizer, callback_manager=callback_manager)
'''
query_engine = index.as_query_engine(
    retriever_mode=ListRetrieverMode.EMBEDDING
    , similarity_top_k=3
)
'''

response = query_engine.query(
    "虹夏ちゃんのお姉さんの仕事は？"
)
'''
response = index.query(
    "虹夏ちゃんのお姉さんの仕事は？", 
    mode="embedding", 
    verbose=True, 
    embed_model=embed_model,
    text_qa_template=default_prompt,
    refine_template=refine_prompt,
    similarity_top_k=2
)
'''
response.response


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


start_trace
event_type = query (start)
event_type = retrieve (start)
event_type = embedding (start)
event_type = embedding (end)
event_type = retrieve (end)
event_type = synthesize (start)
event_type = llm (start)
Q:虹夏ちゃんのお姉さんの仕事は？


Context:
文脈情報は以下です。
---
へと呼び方を改める。同じ学校のひとりのことを気にかけるが、一方でひとりの性格などを歯に衣着せず述べ、無自覚のうちに精神的ダメージを与えることもしばしば。そのストレートな物言いは虹夏（アニメではリョウ）に「ナチュラルに鬼畜」と評された。両親は共に公務員であり、曰く「母親はしっかりしている」とのこと。 伊地知 星歌（いじち せいか） 声 - 内田真礼 虹夏の姉で、ライブハウス「STARRY」の店長。ク
とは公にはされていない。 伊地知 虹夏（いじち にじか） 声 - 鈴代紗弓 誕生日：5月29日 / 身長：154cm / 体重：48kg / 血液型：A型 ドラム担当。下北沢高校2年→3年。明るく世話焼きなバンドのリーダーでまとめ役。また、本作におけるツッコミ役。水玉のリボンがトレードマークで、たいてい身体のどこかに身につけている。姉の星歌がライブハウスの店長を務める元バンドマンであることの影響か
ドを始めようと考え、虹夏と喜多に付き合ってもらってギターを購入。以後は学校でもバイト先でも、ひとりにギターを教えてもらおうとまとわりつく。 日向 恵恋奈（ひなた えれな） 作中2年目から登場。16歳で通信制高校在籍の1年生。猫々と同じくSTARRYの新人バイト。一人称は「えれ」。もとは結束バンドと対バンした地下アイドルグループ「天使のキューティクル」のメンバーで、「ラファエル」を名乗っていた。 天
らは「だめな大人」の一人と思われている。 上村 レコーディングエンジニア。朗らかで話し好きな女性。隙あらば料金を上乗せしようとする。 リナ 星歌の元バンドメンバーでドラマー。現在はスタジオミュージシャン。小学生だった虹夏にねだられ、最初にドラムを教えた人物。 星歌に呼ばれ、アル

'ハウスの店長を務める元バンドマンであること'

In [101]:
response = query_engine.query(
    "後藤ひとりがギターに熱中するようになった理由になった出来事は？", 
)

response.response

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


start_trace
event_type = query (start)
event_type = retrieve (start)
event_type = embedding (start)
event_type = embedding (end)
event_type = retrieve (end)
event_type = synthesize (start)
event_type = llm (start)
Q:後藤ひとりがギターに熱中するようになった理由になった出来事は？


Context:
文脈情報は以下です。
---
ているといち早く気付いている。 先述の性格に加え、運動や勉強など取り柄と言えるものがないというコンプレックスを持っていたところ、中学1年時に暗い学生時代から一転してスターとなったバンドマンのインタビューを目にしたことで、父親の直樹から借りたギターに没頭する。高校入学後はギターを持って公園にいたところを虹夏から声を掛けられ、逃げた郁代の代わりとして臨時で結束バンドのギターを務める運びになり、その流れ
島淳司 誕生日：10月16日 / 血液型：O型 後藤姉妹の父。洋楽趣味でギターを所持しており、ひとりは当初そのギターを借りて使用していた。ひとりの音楽活動については「洋楽にハマってほしかった」としながらも好意的に見守っており、文化祭ライブでギターを壊したことを告白された際には「ライブ中に壊すなんて興奮しちゃうなあ」と話すなどロックに対する理解も高いが、当時中1だったひとりに貸し与えたギターは、虹夏
バンドに厳しくあたることもあるが、実際には誰よりも結束バンドのことを親身に考えており、特に虹夏の事で一喜一憂する。ぶっきらぼうな態度や性格に似合わず可愛い物が好きで、ぬいぐるみを抱いていたりメイド服や女子学生服などを鑑賞用に所持していたりする。また涙もろい。 ひとりのギターヒーローとしての正体まではわからなかったものの、ひとりのギタリストとしての才能にもいち早く気づいており、ひとりを中心にした結束
うになった。また、きくりとの出会いや様々なライブを経て、人間的に成長していく。 毎日6時間以上ギターの練習を約3年間欠かさずやってきたため、プロレベルと称されるほど個人としての演奏技術はずば抜けて高い。一方で人前が苦手な性格や、他人と合わせながら

'夏バンド'

In [102]:
response = query_engine.query(
    "ギターヒーローの正体は誰？", 
)
response.response


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


start_trace
event_type = query (start)
event_type = retrieve (start)
event_type = embedding (start)
event_type = embedding (end)
event_type = retrieve (end)
event_type = synthesize (start)
event_type = llm (start)
Q:ギターヒーローの正体は誰？


Context:
文脈情報は以下です。
---
女子学生服などを鑑賞用に所持していたりする。また涙もろい。 ひとりのギターヒーローとしての正体まではわからなかったものの、ひとりのギタリストとしての才能にもいち早く気づいており、ひとりを中心にした結束バンドの動画を密かに撮影するなど注目している。 PAさん 声 - 小岩井ことり ミステリアスな風貌をしたSTARRYの女性PAエンジニア。最終学歴は高校中退。プライベートでは「音戯 アルト（おとぎ ア
束バンドの取材にSTARRYを訪れ、演奏の癖からひとりとギターヒーローが同一人物であることを見抜く。ギターヒーローの動画に注目する1人で、その実力を既にプロでも通用すると評価している。一方で結束バンドについては「高校生にしたらレベルはまぁ高いと思う」としつつも、その活動姿勢を「ガチじゃない」「本気でプロを目指しているバンドに見えない」と評し「こんなところでうだうだやってると才能腐っちゃいますよ」と
の子だったが、担当編集から「もっとガチにした方がいい」と指摘を受けて修正していき、たちまち現在の設定となった。 ギターヒーロー ひとりが動画投稿サイトで活動する際に用いるハンドルネーム。ひとりが普段人前に出せないが内面に抱えている承認欲求の象徴とも言えるもので、現実のひとりからはかけ離れた「バスケ部エースの彼氏持ち」や「ラインの友達数は1000人超え」といった設定がつけられている。顔を隠すアングル
スまで結束バンドを取材しに来る。ぽいずん♡やみはそこでのひとりによる演奏のクセを聞いて、ひとりがギターヒーローであることを確信し皆にバラした。ひとり（ギターヒーロー）の腕前なら確実にプロとしても通用すると絶賛した一方、他の結束バンドのメンバーに関しては「”ガチ”じゃない」「本気でプ

'ひとりのギターヒーローとしての正体まではわからなかったものの、ひとりのギタリストとしての才能にもいち早く気づいており、ひとりを中心にした結束バンドの動画を密かに撮影するなど注目している。PAさん声-小岩井ことりミステリアスな風貌をしたSTARRYの女性PAエンジニア。最終学歴は高校中退。プライベートでは「音戯アルト(おとぎア束バンドの取材にSTARRYを訪れ、演奏の癖からひとりとギターヒーローが同一人物であることを'

In [103]:
response = query_engine.query(
    "喜多郁代の両親の仕事は？", 
)
response.response

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


start_trace
event_type = query (start)
event_type = retrieve (start)
event_type = embedding (start)
event_type = embedding (end)
event_type = retrieve (end)
event_type = synthesize (start)
event_type = llm (start)
Q:喜多郁代の両親の仕事は？


Context:
文脈情報は以下です。
---
 声 - 末柄里恵 誕生日：4月3日 /血液型：A型 後藤姉妹の母。ひとり及びふたりと同じくピンク色の髪をしている。人見知りなひとりを何かと心配している。喜多が家に来たときは場を盛り上げようと、喜んで学生コスプレまでした（21年ぶりに制服を着たとのこと）。結束バンドを夫と一緒になって応援しており、そのためにも学生に扮したことがある。 後藤 ふたり（ごとう ふたり） 声 - 和多田美咲 誕生日：7月
たお金を与えている。 目はいつも何か（髪の毛や、ふたりの手など）に隠されていて描かれていない。 会社では窓際族と説明しているが、自宅は金沢八景（横浜市）の戸建てである。 後藤 美智代（ごとう みちよ） 声 - 末柄里恵 誕生日：4月3日 /血液型：A型 後藤姉妹の母。ひとり及びふたりと同じくピンク色の髪をしている。人見知りなひとりを何かと心配している。喜多が家に来たときは場を盛り上げようと、喜んで
 小岩井ことり 後藤家で飼われている犬。直樹によって洋楽由来で名付けられた。初対面の相手にもよくなつく。 リョウの両親 職業は病院の経営。リョウを溺愛しているが、本人からはその愛情を疎ましがられてもいる。だが、バンドのために車を出してもらうなど、リョウは時々頼ってもいる。 結束バンドのファン1号・2号 声 - 市ノ瀬加那（1号）、島袋美由利（2号） ひとりときくりの即興ライブを見て結束バンドのライ
ン。小学生だった虹夏にねだられ、最初にドラムを教えた人物。 星歌に呼ばれ、アルバムレコーディングにてプレッシャーのため実力が出せない虹夏にアドバイスを与えた。 後藤 直樹（ごとう なおき） 声 - 間島淳司 誕生日：10月16日 / 血液型：O型 後藤姉妹の父。洋楽趣味でギターを所持

'。リ'

In [104]:
response = query_engine.query(
    "ひとりが動画投稿する時のハンドルネームは？", 
)
response.response
#contextStr = ""
#for node in llama_debug_handler.get_event_pairs(CBEventType.RETRIEVE):
#  for nodeWithScore in node[1].payload["nodes"]:
#    contextStr += nodeWithScore.node.text
#inference_answer("ひとりが動画投稿する時のハンドルネームは？", contextStr)

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


start_trace
event_type = query (start)
event_type = retrieve (start)
event_type = embedding (start)
event_type = embedding (end)
event_type = retrieve (end)
event_type = synthesize (start)
event_type = llm (start)
Q:ひとりが動画投稿する時のハンドルネームは？


Context:
文脈情報は以下です。
---
の子だったが、担当編集から「もっとガチにした方がいい」と指摘を受けて修正していき、たちまち現在の設定となった。 ギターヒーロー ひとりが動画投稿サイトで活動する際に用いるハンドルネーム。ひとりが普段人前に出せないが内面に抱えている承認欲求の象徴とも言えるもので、現実のひとりからはかけ離れた「バスケ部エースの彼氏持ち」や「ラインの友達数は1000人超え」といった設定がつけられている。顔を隠すアングル
ると、顔を上げ表情も整えればアイドル事務所も狙えるほどのルックスだが、その顔は10秒と持たない。そして自己評価が極めて低く、ひとり自身は外見やスタイルを全く意識していない。 初期設定では普通の可愛い女の子だったが、担当編集から「もっとガチにした方がいい」と指摘を受けて修正していき、たちまち現在の設定となった。 ギターヒーロー ひとりが動画投稿サイトで活動する際に用いるハンドルネーム。ひとりが普段人
前に出せないが内面に抱えている承認欲求の象徴とも言えるもので、現実のひとりからはかけ離れた「バスケ部エースの彼氏持ち」や「ラインの友達数は1000人超え」といった設定がつけられている。顔を隠すアングルから撮影された人気バンドのカバー動画を投稿しており、投稿サイトでの登録者数は8万人近く。ぽいずん♡やみのような音楽関係者からも注目を集めている。動画は直樹によって収益化されており、文化祭ライブで壊れた
バンドの動画を密かに撮影するなど注目している。 PAさん 声 - 小岩井ことり ミステリアスな風貌をしたSTARRYの女性PAエンジニア。最終学歴は高校中退。プライベートでは「音戯 アルト（おとぎ アルト）」の名前でバーチャルYouTuberとして活動しており、ゲー

'ギターヒーローひとりが'